## Append MF Ranking to Explanations

This notebook appends predictions from BiasedMF onto explanations from the IJCAI2017 evaluations.

In [67]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina' 

import os
import sys
import numpy as np
from glob import glob
import os
from pprint import pprint as pp
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import logging

HOME_DIR = os.environ['HOME']
PROJECT_DIR = '{}/projects/cikm2017/'.format(HOME_DIR)

sys.path.append(PROJECT_DIR)
os.chdir(PROJECT_DIR)


import cikm
from cikm import merge_predictions, filter_incomplete_sessions

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)
logger = logging.getLogger(__name__)

LIBREC_RESULT_DIR = '/Users/khalil/projects/librec/result/'


# Paths to extraction files.
TA_EXTRACTION_CSV = '/Users/khalil/data/tripadvisor/extractions-ta.csv'
BA_EXTRACTION_CSV = '/Users/khalil/data/beeradvocate/extractions.csv'
YP_EXTRACTION_CSV = '/Users/khalil/data/yelp/extractions.csv'
CIKM_HDF_DIR = '/Users/khalil/projects/cikm2017/data/hdf5/'
HDF5_DIR = CIKM_HDF_DIR
IJCAI17_DATA_DIR = '/Users/khalil/data/IJCAI-2017-data/'

EXTRACTION_CSVS = {'ta': TA_EXTRACTION_CSV, 
                   'ba': BA_EXTRACTION_CSV, 
                   'yp': YP_EXTRACTION_CSV}

PREDICTION_FILE_FMT = os.path.join(LIBREC_RESULT_DIR, '{dataset}/rating/ratings.txt-{algorithm}-output/{algorithm}')
# EXPLANATION_HDF_TEMPLATE = os.path.join(HDF5_DIR, '{dataset}_bw_{weight}_sessions.h5')
EXPLANATION_HDF_TEMPLATE = os.path.join(IJCAI17_DATA_DIR, '{dataset}_bw_{weight}_sessions.h5')

DATA_DIR = '/Users/khalil/data/IJCAI-2017-data/'
paths = glob(os.path.join(DATA_DIR, '*_sessions.h5'))
algorithm = 'biasedmf'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
paths

['/Users/khalil/data/IJCAI-2017-data/ba_bw_iw_sessions.h5',
 '/Users/khalil/data/IJCAI-2017-data/ba_bw_nw_sessions.h5',
 '/Users/khalil/data/IJCAI-2017-data/ba_bw_uw_sessions.h5',
 '/Users/khalil/data/IJCAI-2017-data/ta_bw_iw_sessions.h5',
 '/Users/khalil/data/IJCAI-2017-data/ta_bw_nw_sessions.h5',
 '/Users/khalil/data/IJCAI-2017-data/ta_bw_uw_sessions.h5',
 '/Users/khalil/data/IJCAI-2017-data/yp_bw_iw_sessions.h5',
 '/Users/khalil/data/IJCAI-2017-data/yp_bw_nw_sessions.h5',
 '/Users/khalil/data/IJCAI-2017-data/yp_bw_uw_sessions.h5']

In [69]:
def extract_dataset_id(path):
    idx = path.rfind('/')
    dataset_id = path[idx + 1:].replace('_sessions.h5', '')
    return dataset_id.replace('df_', '')

In [70]:
dataset_ids = [extract_dataset_id(p) for p in paths]
dataset_ids

['ba_bw_iw',
 'ba_bw_nw',
 'ba_bw_uw',
 'ta_bw_iw',
 'ta_bw_nw',
 'ta_bw_uw',
 'yp_bw_iw',
 'yp_bw_nw',
 'yp_bw_uw']

In [72]:
datasets = {'ta': 'TripAdvisor', 'ba': 'BeerAdvocate', 'yp': 'Yelp'}
for dataset_id in dataset_ids:
    for weight in ('nw', 'uw', 'iw'):
        dataset = dataset_id[:2]
        
        print('Processing {}, {}'.format(dataset_id, dataset))
        print(datasets.keys(), datasets[dataset])
        
        prediction_csv_path = PREDICTION_FILE_FMT.format(dataset=datasets[dataset],
                                                         algorithm=algorithm)
        explanation_hdf_path = EXPLANATION_HDF_TEMPLATE.format(dataset=dataset, 
                                                               weight=weight)
        df_explanations = merge_predictions(extraction_csv_path=EXTRACTION_CSVS[dataset], 
                                            prediction_csv_path=prediction_csv_path, 
                                            algorithm=algorithm, 
                                            explanation_hdf5_path=explanation_hdf_path)
        n_unrated = len(df_explanations[pd.isnull(df_explanations['rank_{}'.format(algorithm)])].explanation_id.unique())
        print('Explanations without ratings: {:,}'.format(n_unrated))
        
        # Save to the CIKM data directory using the same filename.
        hdf_filename = explanation_hdf_path[explanation_hdf_path.rfind('/') + 1:]
        output_path = os.path.join(CIKM_HDF_DIR, hdf_filename)
        df_explanations = filter_incomplete_sessions(df_explanations)
        df_explanations.to_hdf(path_or_buf=output_path, mode='w', complevel='blosc', key='index_session', 
                               format="fixed")


INFO:Loading extractions from /Users/khalil/data/beeradvocate/extractions.csv


Processing ba_bw_iw, ba
dict_keys(['ta', 'ba', 'yp']) BeerAdvocate
len(df_ratings)  131702
/Users/khalil/projects/librec/result/BeerAdvocate/rating/ratings.txt-biasedmf-output/biasedmf
Loaded 325,640 explanations
Explanations without ratings: 0


,better_avg,better_avg_comp,better_count,better_pro_scores,cons,cons_comp,explanation_id,is_comp,is_seed,n_cons,...,rank_strength_0.8_ndcg,rank_related_items_sims_np_ndcg,rank_strength_ndcg,rank_strength_comp_ndcg,rank_target_item_average_rating_ndcg,strength_0.5,rank_strength_0.5,rank_strength_0.5_ndcg,rating_biasedmf,rank_biasedmf
0,0.000000,0.000000,"[2, 3, 1, 5, 0, 4]","[0.222222222222, 0.333333333333, 0.11111111111...","[True, True, True, True, True, True]","[True, True, True, False, True, True]",mitchster#12830##11,True,False,6,...,0.947394,0.891345,0.865215,0.947394,1.0,-2.166667,10,0.865215,1.500000,8.0
1,0.777778,0.777778,"[3, 7, 7, 1, 0, 1]","[0.333333333333, 0.777777777778, 0.77777777777...","[True, False, False, True, True, True]","[True, False, False, True, True, True]",mitchster#12830##8782,True,False,4,...,0.934623,0.903105,0.915640,0.934623,1.0,-0.944444,7,0.915640,2.974151,6.0


/Users/khalil/anaconda2/envs/cikm2017/lib/python3.6/site-packages/pandas/core/generic.py:1282: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block5_values] [items->['better_count', 'better_pro_scores', 'cons', 'cons_comp', 'explanation_id', 'pros', 'pros_comp', 'seed_item_id', 'session_id', 'target_item_id', 'target_item_mentions', 'target_item_sentiment', 'user_id', 'worse_con_scores', 'worse_count']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
INFO:Loading extractions from /Users/khalil/data/beeradvocate/extractions.csv


Processing ba_bw_iw, ba
dict_keys(['ta', 'ba', 'yp']) BeerAdvocate
len(df_ratings)  131702
/Users/khalil/projects/librec/result/BeerAdvocate/rating/ratings.txt-biasedmf-output/biasedmf
Loaded 325,640 explanations
Explanations without ratings: 0


...

Processing yp_bw_uw, yp
dict_keys(['ta', 'ba', 'yp']) Yelp
len(df_ratings)  294657
/Users/khalil/projects/librec/result/Yelp/rating/ratings.txt-biasedmf-output/biasedmf
Loaded 56,930 explanations
Explanations without ratings: 0


,better_avg,better_avg_comp,better_count,better_pro_scores,cons,cons_comp,explanation_id,is_comp,is_seed,n_cons,...,rank_strength_0.4,rank_strength_0.4_ndcg,strength_0.6,rank_strength_0.6,rank_strength_0.6_ndcg,strength_0.7,rank_strength_0.7,rank_strength_0.7_ndcg,rating_biasedmf,rank_biasedmf
0,0.160494,0.08642,"[6, 4, 7, 4, 0, 6, 6, 0]","[0.666666666667, 0.444444444444, 0.77777777777...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...",4PY4uUcqPG9oD4c6Uvnw_g#1MMztCZKPtw9DoCVflfZFg#...,True,False,0,...,4,1.00000,0.288889,4,1.00000,0.337037,4,1.00000,3.938692,5.0
1,0.000000,0.00000,"[3, 2, 1, 2, 9, 5, 1, 7]","[0.333333333333, 0.222222222222, 0.11111111111...","[False, True, True, True, False, False, False,...","[False, True, True, True, False, False, False,...",4PY4uUcqPG9oD4c6Uvnw_g#1MMztCZKPtw9DoCVflfZFg#...,True,True,3,...,9,0.87914,-0.316049,9,0.87914,-0.237037,9,0.87914,1.000000,7.0


INFO:Loading extractions from /Users/khalil/data/yelp/extractions.csv


Processing yp_bw_uw, yp
dict_keys(['ta', 'ba', 'yp']) Yelp
len(df_ratings)  294657
/Users/khalil/projects/librec/result/Yelp/rating/ratings.txt-biasedmf-output/biasedmf
Loaded 56,930 explanations
Explanations without ratings: 0


,better_avg,better_avg_comp,better_count,better_pro_scores,cons,cons_comp,explanation_id,is_comp,is_seed,n_cons,...,rank_strength_0.4,rank_strength_0.4_ndcg,strength_0.6,rank_strength_0.6,rank_strength_0.6_ndcg,strength_0.7,rank_strength_0.7,rank_strength_0.7_ndcg,rating_biasedmf,rank_biasedmf
0,0.123874,0.144144,"[0, 7, 3, 8, 0, 5, 3, 0]","[0.0, 0.777777777778, 0.333333333333, 0.888888...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...",q0lNurXi_ivSWYq-FHYptg#jB1fFl1eIEPYe5FLpCZ5Jg#...,True,False,0,...,6,0.697659,0.297297,6,0.697659,0.346847,6,0.697659,2.736196,10.0
1,0.262346,0.262346,"[0, 5, 9, 1, 0, 0, 6, 0]","[0.0, 0.555555555556, 1.0, 0.111111111111, 0.0...","[False, False, False, True, False, False, Fals...","[False, False, False, True, False, False, Fals...",q0lNurXi_ivSWYq-FHYptg#jB1fFl1eIEPYe5FLpCZ5Jg#...,True,False,1,...,5,0.790066,0.409259,3,0.790066,0.503704,2,0.922520,4.393522,3.0


In [73]:
# for dataset_sn, dataset_ln in zip(['ta', 'ba', 'yp'], ['tripadvisor', 'beeradvocate', 'yelp']):
#     tmpl = 'Processing {dataset_ln} dataset (algorithm: {algo})'

#     prediction_csv_path=PREDICTION_FILE_FMT.format(dataset=dataset_ln, algorithm=algorithm)
#     explanation_hdf_path=EXPLANATION_HDF_TEMPLATE.format(dataset_sn)

#     df_explanations = merge_predictions(extraction_csv_path=EXTRACTION_CSVS[dataset_sn], 
#                                         prediction_csv_path=prediction_csv_path, 
#                                         algorithm=algorithm, 
#                                         explanation_hdf_path=explanation_hdf_path)
# #         print('Columns: ', df_explanations.columns)
#     n_unrated = len(df_explanations[pd.isnull(df_explanations['rank_{}'.format(algorithm)])].explanation_id.unique())
#     print('Explanations without ratings: {:,}'.format(n_unrated))

#     # Save to the CIKM data directory using the same filename.
#     hdf_filename = explanation_hdf_path[explanation_hdf_path.rfind('/') + 1:]
#     output_path = os.path.join(CIKM_HDF_DIR, hdf_filename)
#     df_explanations = filter_incomplete_sessions(df_explanations)
#     df_explanations.to_hdf(path_or_buf=output_path, mode='w', complevel='blosc', key='index_session', 
#                            format="fixed")

## Compute NDCGs

In [74]:
glob(CIKM_HDF_DIR + '*._sessions')

['/Users/khalil/projects/cikm2017/data/hdf5/ba-bw-nw-a00-sessions.h5',
 '/Users/khalil/projects/cikm2017/data/hdf5/ba_bw_iw_sessions.h5',
 '/Users/khalil/projects/cikm2017/data/hdf5/ba_bw_nw_sessions.h5',
 '/Users/khalil/projects/cikm2017/data/hdf5/ba_bw_uw_sessions.h5',
 '/Users/khalil/projects/cikm2017/data/hdf5/ta-bw-nw-a00-sessions.h5',
 '/Users/khalil/projects/cikm2017/data/hdf5/ta_bw_iw_sessions.h5',
 '/Users/khalil/projects/cikm2017/data/hdf5/ta_bw_nw_sessions.h5',
 '/Users/khalil/projects/cikm2017/data/hdf5/ta_bw_uw_sessions.h5',
 '/Users/khalil/projects/cikm2017/data/hdf5/yp-bw-nw-a00-sessions.h5',
 '/Users/khalil/projects/cikm2017/data/hdf5/yp_bw_iw_sessions.h5',
 '/Users/khalil/projects/cikm2017/data/hdf5/yp_bw_nw_sessions.h5',
 '/Users/khalil/projects/cikm2017/data/hdf5/yp_bw_uw_sessions.h5']

In [76]:
NDCG_DIR = '/Users/khalil/projects/cikm2017/data/ndcg/'
for path in glob(CIKM_HDF_DIR + '*w_sessions.h5'):
    df_explanations = pd.read_hdf(path)

    df_ndcgs = cikm.compute_ndcg(df_explanations)
    df_ndcgs_summary = cikm.summarize_ndcg(df_ndcgs)
    suffix = path[path.rfind('/')+1:].replace('.h5', '')
    save_path = os.path.join(NDCG_DIR, 'ndcg-summary-{}.csv'.format(suffix))

    df_ndcgs_summary.to_csv(save_path, index=False)